In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [2]:
# Load data and transform date columsn to "datetime"
raw_data = pd.read_csv("../../data-ProPublica/compas-scores.csv", parse_dates=["c_jail_in", "c_jail_out"])
raw_data.shape

(11757, 47)